In [28]:
import json
import pandas as pd

import portrisk

from deribit import get_order_book

pd.set_option('display.max_rows', None)  # Set max_rows to None to display all rows
pd.set_option('display.max_columns', None)  # Set max_columns to None to display all columns


In [34]:
# Load stress parameters
file_path = './parameters/'

with open(file_path + 'spot_vol_shocks.json', 'r') as f:
    spot_vol_shocks_param = json.load(f)

crypto_parameters = portrisk.core.utils.CryptoParameters(spot_vol_shocks_param)

with open(file_path + 'vol_surface_shocks.json', 'r') as f:
    vol_surface_shocks_param = json.load(f)

vol_surface_shocks_parameters = portrisk.core.utils.VolSurfaceParameters(vol_surface_shocks_param)


In [64]:
# load position data
df = pd.read_excel('positions.xlsx')

# Splitting the 'instrument' column by "-"
df[['underlying','expiry','strike','put_call']] = df['instrument'].str.split('-', expand=True)
df['strike'] = df['strike'].astype(float)
df['put_call'] = df['put_call'].map({'P':'put','C':'call'})
df['expiry'] = pd.to_datetime(df['expiry'], format='%d%b%y')

# Calculate time to expiry
valuation_day = pd.to_datetime('2024-08-05')
df['time_to_expiry'] = (df['expiry'] - valuation_day).dt.days / 365

# get market data from deribit
deribit_res = get_order_book(instruments=df['instrument'].to_list())
df = pd.merge(df, deribit_res,how='left',left_on='instrument',right_on='instrument_name')


# calculate input for BSM		
df['cost_of_carry_rate'] = 'default'
df['multiplier'] = 1
df['rate'] = 0.03
df['vol'] = df['mark_iv'] / 100
df['spot'] = df['index_price'].values[0]

df['PositionVega'] = df['vega'] * df['quantity']
df['Expiry'] = df['expiry']

## Spot Vol Shocks

In [66]:
spot_vol_shocks = portrisk.crypto.CryptoSpotVolShocks(crypto_parameters)

In [78]:
post_stress, col = spot_vol_shocks.apply_spot_vol_shocks(df)

In [23]:
portrisk.core.black_scholes.bs_pricing(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')
portrisk.core.black_scholes.calc_vega(strike=50000, time_to_expiry=0.232877, spot=54681.59, rate=0.03, vol=0.5775, put_call='put')

# np.float64(3606.873391368419)
# np.float64(3700.518323796012)

np.float64(93.56524889234046)

In [ ]:
post_stress

## Vol Surface Shocks

In [35]:
vol_surface_shocks_parameters.config_concentration('BTC')

{'Class': 'BTC',
 'alpha_r': 0.25,
 'beta_r': 0.42482,
 'alpha_a': 12.8,
 'beta_a': 0.275641,
 'max_vol': 70.18,
 'min_vol': 13.07,
 '1m': 1000000,
 '3m': 1500000,
 '6m': 2000000,
 '1y': 3000000,
 '2y': 1000000}

In [33]:
vol_surface_shocks_parameters.parameters

{'scenario_1': {'spot_shock': {'BTC': 0.1, 'ETH': 0.2},
  'vol_shock': {'BTC': 0, 'ETH': 0}},
 'scenario_2': {'spot_shock': {'BTC': 0.15, 'ETH': 0.25},
  'vol_shock': {'BTC': 0, 'ETH': 0}}}